In [5]:
!pip3 install --upgrade pandas


     |████████████████████████████████| 11.3 MB 1.5 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Uninstalling pandas-1.2.5:
      Successfully uninstalled pandas-1.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.8.3 requires cupy-cuda114, which is not installed.
cudf 21.8.3 requires cupy-cuda110, which is not installed.
caip-notebooks-serverextension 1.0.0 requires google-cloud-bigquery-storage, which is not installed.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.3 which is incompatible.
dask-cudf 21.8.3 requires dask<=2021.07.1,>=2021.6.0, but you have dask 2021.9.0 which is incompatible.
dask-cudf 21.8.3 requires distributed<=2021.07.1,>=2021.6.0, but you have distributed 2021.9.0 which is incompatible.
dask-cudf 21.8.3 requires pandas<1.3.0dev0,>=1.0, but you have p

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import  numpy as np
import torch
import collections

from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix,f1_score, accuracy_score,precision_score, recall_score
from sklearn.model_selection import ParameterGrid
import itertools
# testdata=[]

# valdata=[]
import os
# for dirname, _, filenames in os.walk('./predictions'):
#     for filename in filenames:
#         if filename.endswith((".pkl",".tsv")):
#             if "Valpred" in filename:
#                 valdata.append(os.path.join(dirname, filename))
#             elif "Testpred" in filename:
#                 testdata.append(os.path.join(dirname, filename))
#             else:
#                 print(os.path.join(dirname, filename))
                
# testdfs=[torch.load(x) for x in testdata]
# valdfs=[torch.load(x) for x in valdata]
testpredictions=pd.DataFrame()


def getexcludelist(dfs):
    """ exclude not duplicate suid  """
    valdfsSuid=[df.suid for df in dfs]
    excludelist=[]
    for i,lst in enumerate(valdfsSuid):
        othlst=set()
        for j,o in enumerate(valdfsSuid):
            if j!=i:
                othlst.update(o)
        excludelist.extend(list(set(lst).symmetric_difference(set(othlst))))
    excludelist=set(excludelist)
    print("excludelist" ,len(excludelist))
    return excludelist

def getConfsid(tdfsinp):
    """      get ids which has high confidence.    """
    intersectIdxinfolds=[]
    for fold in range(1,6):
        for df in tdfsinp:
            for fld in range(1,6):
                df[f'Tconf{fld}']=df[f'confidence{fld}'].apply(lambda x:np.amax(x)) 
                df[f'T2confs{fld}']=df[f'confidence{fld}'].apply(lambda x:x[(-x).argsort()[:2]]) 
                df[f'T2condif{fld}']=df[f'T2confs{fld}'].apply(lambda x:x[0]-x[1])
        
        tdfs=[df[(df[f'predictions{fold}']!=0) & (df[f'T2condif{fold}']>10)& (df[f'Tconf{fold}']>90)]  for df in tdfsinp]
        tdfs=[df.sid.tolist() for df in tdfs]
        intersectIdxinfolds.append(set.intersection(*map(set,tdfs)))
    print([len(x) for x in intersectIdxinfolds])
    combinelist=list(itertools.chain.from_iterable(intersectIdxinfolds))
    print(len(combinelist))
    return (combinelist)





def cleanvaldf(vdfs):
    """      remove duplicate    """
    print("vdfs len before removing label 0:",[len(df.index) for df in vdfs])
    
    vdfs=[df[df.label!=0] for df in vdfs]

    print("should have same lenght: ",[len(df.index) for df in vdfs])
    
    excludelist=getexcludelist(vdfs)
    print("check nonduplicate:" ,len(excludelist))

    print(excludelist)
    
    vdfs=[df[~df.suid.isin(excludelist)] for df in vdfs]
    print("removed nonduplicate")
    vdfs=[df.sort_values(["html_id", "sid"], ascending = (True, True)) for df in vdfs]
    
    print("Same labels:",[df.label.tolist()==vdfs[0].label.tolist()  for df in vdfs])
    
    return vdfs

def getParams(df):
    """ weight parameter grids """
    param_grid={}
    for i in range(len(valdfs)):
        param_grid['wt'+str(i)]=list(range(0,10)) 
    grid = ParameterGrid(param_grid)
    return grid


def essembleValidate(dfs,grid):
    """ get the best predictions (for pseudo lable)"""
    label= np.array(dfs[0]['label'].tolist())
    preds=[df[[col for col in df if col.startswith('logits')][0]].tolist() for df in dfs]
    preds=[pd.DataFrame(np.array(p), columns=['0','1','2','3']) for p in preds]
    df = pd.DataFrame([])
    preds=np.array([p.to_numpy() for p in preds])
    for params in grid:
        wts = list(params.values())
        wted_preds = np.tensordot(preds, wts, axes=((0),(0)))
        wted_ensemble_pred = np.argmax(wted_preds, axis=1)
        f1score= f1_score(label, wted_ensemble_pred, average='weighted')   
        params['f1score']=f1score
        df = df.append(pd.DataFrame(params, index=[0]), ignore_index=True)
    
    max_acc_row = df.iloc[df['f1score'].idxmax()]
    print("pred1logitCol:",[[col for col in df if col.startswith('logits')][0] for df in dfs])
    display(max_acc_row)
    del max_acc_row["f1score"]
    return max_acc_row.tolist()
    
def essembleTest(tdfs,wts,fold):
    """ get weighted essemble preditions for test"""

    preds=[df[[col for col in df if col.startswith('logits'+str(fold))][0]].tolist() for df in tdfs]
    preds=[pd.DataFrame(np.array(p), columns=['0','1','2','3']) for p in preds]
    preds=np.array([p.to_numpy() for p in preds])
    wted_preds = np.tensordot(preds, wts, axes=((0),(0)))
    wted_ensemble_pred = np.argmax(wted_preds, axis=1)
#     print("predictions summary: ",collections.Counter(wted_ensemble_pred))
    return wted_ensemble_pred


In [39]:
testdata=[]
valdata=[]
for dirname, _, filenames in os.walk('./predictions_PLF'):
    for filename in filenames:
        if filename.endswith((".pkl")):
            if "Valpred" in filename:
                valdata.append(os.path.join(dirname, filename))
            elif "Testpred" in filename:
                testdata.append(os.path.join(dirname, filename))
            else:
                print(os.path.join(dirname, filename))
    
# print("testdata")
# display(testdata)
print(testdata)
# testdfs=[torch.load(x) for x in testdata]
# print("Test size:",len(testdfs[0].index))
# confSid=getConfsid(testdfs)
# print("High confident size:",len(confSid))
# valdfs=[torch.load(x) for x in valdata]
# print("valdata")
# display(valdata)

['./predictions_PLF/roberta/rFNTestpred.pkl', './predictions_PLF/deberta/DbcawP1/dFNTestpreds.pkl', './predictions_PLF/albert/AbcawP1/aFNTestpreds.pkl', './predictions_PLF/robertapool/rFNTestpreds.pkl']


In [41]:
rp=torch.load('./predictions_PLF/roberta/rFNTestpred.pkl')
ap=torch.load('./predictions_PLF/albert/AbcawP1/aFNTestpreds.pkl')
dp=torch.load('./predictions_PLF/deberta/DbcawP1/dFNTestpreds.pkl')
rpp=torch.load('./predictions_PLF/robertapool/rFNTestpreds.pkl')

oap=torch.load('./predictions_Otrain/deberta/Dbca/dTestpreds.pkl')
odp=torch.load('./predictions_Otrain/albert/Abca/aTestpreds.pkl')

rp=rp.sort_values(['sid'], ascending=True)
ap=ap.sort_values(['sid'], ascending=True)
dp=dp.sort_values(['sid'], ascending=True)
rpp=rpp.sort_values(['sid'], ascending=True)
oap=oap.sort_values(['sid'], ascending=True)
odp=odp.sort_values(['sid'], ascending=True)



In [45]:
summed=np.sum([
              np.array(rp.logits1.values.tolist())*0.4,
              np.array(rp.logits2.values.tolist())*0.4,
              np.array(rp.logits3.values.tolist())*0.4,
              np.array(rp.logits4.values.tolist())*0.4,
              np.array(rp.logits5.values.tolist())*0.4,

              np.array(ap.logits1.values.tolist())*0.1,
              np.array(ap.logits2.values.tolist())*0.1,
              np.array(ap.logits3.values.tolist())*0.1,
              np.array(ap.logits4.values.tolist())*0.1,
              np.array(ap.logits5.values.tolist())*0.1,
    
              np.array(dp.logits1.values.tolist())*0.1,
              np.array(dp.logits2.values.tolist())*0.1,
              np.array(dp.logits3.values.tolist())*0.1,
              np.array(dp.logits4.values.tolist())*0.1,
              np.array(dp.logits5.values.tolist())*0.1,
    
              np.array(rpp.logits1.values.tolist())*0.2,
              np.array(rpp.logits2.values.tolist())*0.2,
              np.array(rpp.logits3.values.tolist())*0.2,
              np.array(rpp.logits4.values.tolist())*0.2,
              np.array(rpp.logits5.values.tolist())*0.2,
    

    ],axis=0)

finalpreds=np.argmax(summed,axis=1)

cols=[c for c in ap.columns if c.startswith("prediction")]


rp=rp[['sid','sentence','predictions1',"predictions2","predictions3","predictions4","predictions5"]]
rp['final']=finalpreds
rp[['al1','al2','al3','al4','al5']]=ap[cols]
rp[['dl1','dl2','dl3','dl4','al5']]=dp[cols]


In [48]:
submission=rp[['sid','final']]
submission.to_csv('final_submission6.tsv', sep='\t', header=False, index=False)
rp.to_csv('final_submission5.tsv', sep='\t', header=False, index=False)
rp.to_csv('debugfinal_3.tsv', sep='\t', index=False)

In [20]:
ap[cols]

,logits1,logits2,logits3,logits4,logits5
6656,"[3.0802, -1.9553826, -1.4889772, -0.8823516]","[3.7948003, -1.2423111, -0.84380215, -0.3480488]","[4.380255, -1.8481692, -1.3575437, -0.64343905]","[3.1739135, -1.39625, -1.5155038, -0.055402778]","[3.6735044, -2.1300135, -1.6872667, -0.82474154]"
6657,"[3.2422812, -2.0478413, -1.6866279, -0.75599223]","[4.5924973, -1.7184323, -1.2202939, -0.52553594]","[4.722586, -2.3896391, -1.8665965, -0.7205733]","[4.1816897, -1.9028848, -1.4969223, -0.13259931]","[4.6876187, -2.5973516, -2.1072855, -0.4646309]"
6658,"[3.2379122, -2.211926, -1.739107, -0.6793136]","[4.5475307, -1.8399135, -1.1798309, -0.25535828]","[4.711764, -2.5420754, -1.7365987, -0.5063674]","[4.2336035, -1.7477938, -1.4764534, -0.34303597]","[4.402654, -2.565669, -1.8920058, -0.42559606]"
6659,"[3.2471159, -1.9556475, -1.4725784, -1.0927179]","[4.24002, -1.4342341, -1.2670465, -0.4112149]","[4.300529, -2.3883548, -1.5894133, -0.4265851]","[2.3494687, -1.3481914, -1.7185758, 0.3752105]","[4.3096967, -2.3735752, -1.8924637, -0.8510866]"
6660,"[2.105722, -1.3781784, -1.2939456, 0.060199153]","[2.087596, -0.7898353, -0.2783787, 0.81896484]","[3.1423414, -1.9518927, -1.0925063, 0.70627433]","[1.8599517, -1.2796087, -1.5876671, 0.5923834]","[1.476131, -1.449832, -1.1187595, 0.32263285]"
...,...,...,...,...,...
60502,"[3.353171, -2.042561, -1.5702131, -1.0446868]","[4.6403313, -1.7945074, -1.4155688, -0.37161225]","[4.5747943, -2.5091639, -1.8119781, -0.4279667]","[3.8125699, -1.6652552, -1.4479547, -0.25614786]","[4.6241746, -2.353856, -2.0192914, -1.2065104]"
60503,"[3.3018887, -2.1218963, -1.4532712, -1.0221975]","[4.650549, -1.7644849, -1.3710417, -0.48334587]","[4.18614, -2.5036726, -1.6355969, -0.20441051]","[3.6653295, -1.5191714, -1.2911607, -0.35773224]","[4.793032, -2.3988504, -1.8957782, -1.0807381]"
60504,"[3.2770944, -1.9039315, -1.444788, -1.0983943]","[4.2270856, -1.4035306, -1.2292924, -0.31876713]","[4.368389, -2.390686, -1.6310568, -0.38876873]","[2.1089015, -1.4324557, -1.7062415, 0.62701416]","[3.798963, -2.2328637, -1.7910721, -0.67908597]"
60505,"[3.3830113, -1.9724435, -1.565154, -0.900899]","[4.2823515, -1.3597143, -1.3088282, -0.34006852]","[4.4477043, -2.5387573, -1.6693977, -0.54397255]","[3.3986833, -1.9604679, -1.8754586, 0.6565382]","[2.6051064, -1.7207667, -1.723918, 0.075070195]"


In [103]:
rp['submission']=finalpreds

In [17]:
cols=testdfs[0].columns
cols=[c for c in testdfs[0].columns if c.startswith("prediction")]
testdfs=[df[cols] for df in testdfs]

In [4]:

# for fold in range(1,6):
for fold in range(1,6):
    print("fold :",fold)
    valdataf=[val for val in valdata if f"pred{fold}" in val]
    print("valdataf")
    display(valdataf)
    valdfs=[torch.load(x) for x in valdataf]
    valdfs=cleanvaldf(valdfs)
    print("should have same lenght: ",[len(df.index) for df in valdfs])
    grid=getParams(valdfs)
    weights=essembleValidate(valdfs,grid)  
    testpredictions[fold]=essembleTest(testdfs,weights,fold)  
    

testpredictions['prediction']=testpredictions.mode(axis=1).iloc[:, 0]
testpredictions['prediction']=testpredictions['prediction'].astype('int32')
column=['suid', 'sid', 'html_id', 'ori_sentence', 'title', 'tid', 'random',
       'asentence', 'atid', 'arandom', 'bsentence', 'btid', 'brandom',
       'sentence']

testpredictions[column]=testdfs[0][column]

# testp=testpredictions[testpredictions.sid.isin(confSid)]
# testp=testp[testp.prediction!=0]
testp=testpredictions
testp['Hconf']=testpredictions.sid.apply(lambda x: x in confSid)
print("All pred count")
print(testpredictions.prediction.value_counts())
print("High conf count")
print(testp.prediction.value_counts())
testp=testp.sort_values(['html_id', 'sid'], ascending=[True, True])
testp=testp.rename({'prediction': 'label'}, axis=1)
testp.drop([1,2,3,4,5], axis=1, inplace=True)
testp.to_csv("ESBtest1predictions.csv")

fold : 1
valdataf


['./predictions_PL1/deberta/Dbca_pl1/dValpred1.pkl',
 './predictions_PL1/deberta/Dbca_pl1 (copy)/dValpred1.pkl']

vdfs len before removing label 0: [1293, 1293]
should have same lenght:  [449, 449]
excludelist 0
check nonduplicate: 0
set()
removed nonduplicate
Same labels: [True, True]
should have same lenght:  [449, 449]
pred1logitCol: ['logits1_f0.853_v0.033_dbca', 'logits1_f0.853_v0.033_dbca']


wt0        0.000000
wt1        1.000000
f1score    0.832427
Name: 1, dtype: float64

fold : 2
valdataf


...

fold : 5
valdataf


['./predictions_PL1/deberta/Dbca_pl1/dValpred5.pkl',
 './predictions_PL1/deberta/Dbca_pl1 (copy)/dValpred5.pkl']

vdfs len before removing label 0: [424, 424]
should have same lenght:  [218, 218]
excludelist 0
check nonduplicate: 0
set()
removed nonduplicate
Same labels: [True, True]
should have same lenght:  [218, 218]
pred1logitCol: ['logits5_f0.83_v0.038_dbca', 'logits5_f0.83_v0.038_dbca']


wt0        0.000000
wt1        1.000000
f1score    0.817788
Name: 1, dtype: float64

All pred count
0    23742
3     3713
2      366
1      181
Name: prediction, dtype: int64
High conf count
0    23742
3     3713
2      366
1      181
Name: prediction, dtype: int64


In [5]:
# columns=['suid','logits1','predictions1','confidence1','sentence']
# df=testdfs[0][columns]
# for fld in range(1,6):
#     df[f'Tconf']=df[f'confidence1'].apply(lambda x:np.amax(x)) 
#     df[f'T2confs1']=df[f'confidence1'].apply(lambda x:x[(-x).argsort()[:2]]) 
#     df[f'T2condif1']=df[f'T2confs1'].apply(lambda x:x[0]-x[1])
# # tdfs=[df[(df[f'predictions1']==1) & (df[f'Tcondif1{fold}']>0)& (df[f'Tconfidence{fold}']>1)]  for df in tdfsinp]
# df

In [6]:
# for df in testdfs:
#     for fld in range(1,6):
#         df[f'Tpreds{fld}']=df[f'confidence{fld}'].apply(lambda x:(-x).argsort()[:2]) 
#         df[f'Tconfs{fld}']=df[f'confidence{fld}'].apply(lambda x:x[(-x).argsort()[:2]]) 
#         df[f'Tcondif{fld}']=df[f'Tconfs{fld}'].apply(lambda x:x[0]-x[1])


In [104]:
# # a = np.array([96.4494288 ,  9.99175012, 22.66806066, 46.49918377])
# # ind = (-a).argsort()[:2]
# # print(ind)
# # a[ind]
# # x=a
testpredictions=rp
testpredictions=testpredictions.sort_values(['sid'], ascending=True)
submission=testpredictions[['sid','submission']]
submission.to_csv('final_submission2.tsv', sep='\t', header=False, index=False)

In [ ]:

rp.to_csv('debugfinal_submission1.tsv', sep='\t')